In [65]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize

In [66]:
doc_1 = "The diagram describes a system architecture in a simple overview. The entity cordinator access the Kafka platform as the the replicas sets do too. Each replica set is composed by two elements: the cluster manager that access the HDFS dabase and the Backend Servers that access the Blobstore database."
doc_2 = "MetricsDB has 3 main components. Cluster Manager is responsible for assigning partitions to backend servers. HDFS is used to store mappings from partitions to servers. Backend Servers are responsible for processing metrics for a small number of partitions. Each Backend Server keeps the latest two hours of data for all metrics in memory and they checkpoint in-memory data every two hours to durable storage, Blobstore.  Coordinators are responsible for routing requests to all replica sets and validates desired quorum constraints are met.  MetricsDB is multi-zone compliant. MetricsDB has a replication factor of three. Apache Kafka producer has an option to batch requests, which helped us reduce the number of requests to the queue and storage."
data = [doc_1, doc_2]

### aplicar lowercase e remover caracteres

In [67]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [68]:
data_clear = clear_text(data)

print('-- Antes --')
print(data)
print()
print('-- Depois --')
print(data_clear)

-- Antes --
['The diagram describes a system architecture in a simple overview. The entity cordinator access the Kafka platform as the the replicas sets do too. Each replica set is composed by two elements: the cluster manager that access the HDFS dabase and the Backend Servers that access the Blobstore database.', 'MetricsDB has 3 main components. Cluster Manager is responsible for assigning partitions to backend servers. HDFS is used to store mappings from partitions to servers. Backend Servers are responsible for processing metrics for a small number of partitions. Each Backend Server keeps the latest two hours of data for all metrics in memory and they checkpoint in-memory data every two hours to durable storage, Blobstore.  Coordinators are responsible for routing requests to all replica sets and validates desired quorum constraints are met.  MetricsDB is multi-zone compliant. MetricsDB has a replication factor of three. Apache Kafka producer has an option to batch requests, which

In [69]:
#word_tokenize(data_clear)

In [70]:
#from nltk.corpus import stopwords

#filtered_words = [t for t in word_tokenize(data_clear) if not t in stopwords.words("english")]

In [71]:
def create_dataframe(matrix, tokens):
    
    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [72]:
Tfidf_vect = TfidfVectorizer()
vector_matrix = Tfidf_vect.fit_transform(data)

tokens = Tfidf_vect.get_feature_names()

create_dataframe(vector_matrix.toarray(),tokens)

,access,all,an,and,apache,architecture,are,as,assigning,backend,...,they,three,to,too,two,us,used,validates,which,zone
doc_1,0.335215,0.000000,0.000000,0.079503,0.000000,0.111738,0.000000,0.111738,0.000000,0.079503,...,0.000000,0.000000,0.000000,0.111738,0.079503,0.000000,0.000000,0.000000,0.000000,0.000000
doc_2,0.000000,0.128597,0.064298,0.137246,0.064298,0.000000,0.192895,0.000000,0.064298,0.137246,...,0.064298,0.064298,0.450088,0.000000,0.091497,0.064298,0.064298,0.064298,0.064298,0.064298


In [73]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['doc_1','doc_2'])

,doc_1,doc_2
doc_1,1.000000,0.185495
doc_2,0.185495,1.000000
